# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config

sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

# import tracking
sys.path.append(TRACKER_DIR) 
from sort.sort import Sort
import glob

# %matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
VIDEO_DIR = '/media/DATA/traffic_accident_videos/images_10hz/'

# indicate GPUs
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench']#, 'bird',
#                'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
#                'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
#                'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
#                'kite', 'baseball bat', 'baseball glove', 'skateboard',
#                'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
#                'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
#                'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
#                'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
#                'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
#                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
#                'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
#                'teddy bear', 'hair drier', 'toothbrush']

num_classes = len(class_names)
num_classes

## Run Object Detection

In [ ]:
'''Run on videos!'''
# del sys.modules['sort']
# del sys.modules['mrcnn']
from mrcnn import visualize
from sort import Sort
from utils import *
import glob
import time

# only for testing
VIDEO_DIR = '/home/yyao/Documents/Future_vehicle_localization/data/videos/'

all_files = glob.glob(VIDEO_DIR+'*.mp4')
# ROI = [0, 0, 1920, 1200]
ROI = [0, 0, 640, 1280]
saver = True
display = True
    
colors = np.random.rand(32, 3)
'''for saving observations of each video'''
all_observations = {}
for video in all_files:
    
    cap = cv2.VideoCapture(video)
    
    '''for display'''
    if display:
        colours = np.random.rand(32, 3)*255  # used only for display
        plt.ion()
        fig = plt.figure()
    
    '''init tracker'''
    use_dlibTracker = False # True to use dlib correlation tracker, False to use Kalman Filter tracker
    all_trackers =  Sort(ROI, max_age=1,min_hits=3, use_dlib=use_dlibTracker,track_masks=True)
    
    '''count time'''
    total_time = 0

#     '''write results'''
    out_path = '/home/yyao/Documents/Future_vehicle_localization/tracking_output/' + video[-22:-4] #+ '/'#[-22:-4]
    out_file = out_path + '_trackings.txt'
    try:
        os.stat(out_file)
        print("video has been processed!")
        
    except:
        aa =1        
    
    f_out = open(out_file, 'w')
    feature_out = '/home/yyao/Documents/Future_vehicle_localization/tracking_output/' + video[-22:-4] + '.npy'
    
    frame = 0

    '''for saving observations of each car'''
    observations = {}
    tracked_features = []
    while(cap.isOpened()):
        ret, img = cap.read()
        
        if img is None:
            break
        img = cv2.resize(img, (1280,800))
        
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)[:640, :]
                
        mrcnn_detections  = model.detect([img], verbose=1)[0]
        
        interesting_objects = np.where(mrcnn_detections['class_ids']<num_classes)[0]
        
        bboxes = mrcnn_detections['rois'][interesting_objects]
        masks = mrcnn_detections['masks'][:,:,interesting_objects]
        classes = mrcnn_detections['class_ids'][interesting_objects]
        scores = mrcnn_detections['scores'][interesting_objects]
        features = mrcnn_detections['roi_features'][interesting_objects]
        
        frame += 1
        start_time = time.time()
        #update tracker
        trackers, feature_list = all_trackers.update(bboxes.astype('int'),
                                                  img=img, 
                                                  masks=masks,
                                                  classes=classes,
                                                  scores=scores,
                                                  features=features) # only cars are considered so far.

        # use correlation tracker
#         trackers, mask_list = all_trackers.update(bboxes.astype('int'),
#                                        img=img,
#                                        classes=classes,
#                                        scores=scores) # only cars are considered so far.
        
        

        cycle_time = time.time() - start_time
        total_time += cycle_time

        print('frame: %d...took: %3fs'%(frame,cycle_time))
        
        tracked_boxes = []
        tracked_id = []
        tracked_masks = []
        tracked_classes = []
        tracked_scores = []
        
        if not use_dlibTracker:
            for j, (d,features) in enumerate(zip(trackers, feature_list)):
                tracked_boxes.append(d[:4])
                tracked_id.append(d[4])
#                 if j == 0:
#                     tracked_masks = mask
#                 else:
#                     tracked_masks = np.dstack([tracked_masks, mask])
    #             tracked_masks.append(mask_list[j])
                tracked_classes.append(d[-1])
                tracked_scores.append(d[-2])
                tracked_features.append(feature_list[j])

                # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
                f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
                            (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))
                
                
                
#             tracked_masks = np.reshape(tracked_masks, (tracked_masks.shape[0],tracked_masks.shape[1],j+1))
        else:
            for j, d in enumerate(trackers):
                tracked_boxes.append(d[:4])
                tracked_id.append(d[4])
                tracked_classes.append(d[-1])
                tracked_scores.append(d[-2])
                tracked_features.append(feature_list[j])
                # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
                f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
                            (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))
        
        tracked_boxes = np.array(tracked_boxes).astype('int')
        tracked_id = np.array(tracked_id)
        if len(tracked_id) == 0:
            continue
        tracked_classes = np.array(tracked_classes).astype('int')
        tracked_scores = np.array(tracked_scores)
#         if frame > 5:
#             break
    
        # save masked images
#         save_path = out_path + str(format(frame,'04'))+'.png'
#         masked_img = visualize.display_tracklets(img,
#                                             tracked_boxes,
#                                             tracked_id,
#                                             tracked_masks, 
#                                             tracked_classes, 
#                                             class_names, 
#                                             tracked_scores,
#                                             show_mask = False,
#                                             colors = colors,
#                                             save_path = save_path)  # used only for display)
    tracked_features = np.array(tracked_features)
    print(tracked_features.shape)
    np.save(feature_out, tracked_features)
    
#         plt.clf()
        
#         save mask files
#         total_mask = np.zeros((640,1280),dtype=bool)
#         for i in range(tracked_masks.shape[2]):
#             total_mask = np.bitwise_or(total_mask, tracked_masks[:,:,i])

#         bbox_mask = np.ones((640,1280))
#         for box in tracked_boxes:
#             bbox_mask[box[0]:box[2], box[1]:box[3]] = 0
#         write_csv(out_path + str(format(frame,'04')) + '.csv' ,total_mask)
    
    print("One video is written!")
    f_out.close()


In [ ]:
folder[-27:-1]

In [ ]:
'''Run on KITTI raw images!'''
# del sys.modules['sort']
# del sys.modules['mrcnn']
from mrcnn import visualize
from sort import Sort
from utils import *
import glob
import time

# only for testing
IMG_DIR = '/home/yyao/Documents/KITTI/KITTI_raw/all_files/'

# all_files = glob.glob(VIDEO_DIR+'*.mp4')
all_folders = glob.glob(IMG_DIR + '*/')

# ROI = [0, 0, 1920, 1200]
W = 1242
H = 375
ROI = [0, 0, 375, 1242]
saver = True
display = True
    
colors = np.random.rand(32, 3)
'''for saving observations of each video'''
all_observations = {}
for folder in all_folders:
    print(folder)    
    '''for display'''
    if display:
        colours = np.random.rand(32, 3)*255  # used only for display
        plt.ion()
        fig = plt.figure()
    
    '''init tracker'''
    use_dlibTracker = False # True to use dlib correlation tracker, False to use Kalman Filter tracker
    all_trackers =  Sort(ROI, max_age=1,min_hits=3, use_dlib=use_dlibTracker,track_masks=True)
    
    '''count time'''
    total_time = 0

#     '''write results'''
    out_path = '/home/yyao/Documents/KITTI/tracking_output/' + folder[-27:-1]# + '/'#[-22:-4]
    out_file = out_path + '_trackings.txt'
    
    feature_out = '/home/yyao/Documents/KITTI/tracking_output/' + folder[-27:-1] + '.npy'    
    try:
        os.stat(out_path)
        print("video has been processed!")
#         continue
    except:
        os.mkdir(out_path)
        aa = 1
    f_out = open(out_file, 'w')
    
    frame = 0
    
    '''for saving observations of each car'''
    observations = {}
    tracked_features = []
    all_images = sorted(glob.glob(folder + '/image_02/data/*.png'))
    for image_file in all_images:
        
        img = cv2.imread(image_file)
        
        if img is None:
            break
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
        mrcnn_detections  = model.detect([img], verbose=1)[0]
        
        interesting_objects = np.where(mrcnn_detections['class_ids']<num_classes)[0]
        
        bboxes = mrcnn_detections['rois'][interesting_objects]
        masks = mrcnn_detections['masks'][:,:,interesting_objects]
        classes = mrcnn_detections['class_ids'][interesting_objects]
        scores = mrcnn_detections['scores'][interesting_objects]
        features = mrcnn_detections['roi_features'][interesting_objects]
        
        frame += 1
        start_time = time.time()
        #update tracker
        trackers, feature_list = all_trackers.update(bboxes.astype('int'),
                                                  img=img, 
                                                  masks=masks,
                                                  classes=classes,
                                                  scores=scores,
                                                  features=features) # only cars are considered so far.

        # use correlation tracker
#         trackers, mask_list = all_trackers.update(bboxes.astype('int'),
#                                        img=img,
#                                        classes=classes,
#                                        scores=scores) # only cars are considered so far.
        
        

        cycle_time = time.time() - start_time
        total_time += cycle_time

        print('frame: %d...took: %3fs'%(frame,cycle_time))
        
        tracked_boxes = []
        tracked_id = []
        tracked_masks = []
        tracked_classes = []
        tracked_scores = []
        
        if not use_dlibTracker:
            for j, (d,features) in enumerate(zip(trackers, feature_list)):
                tracked_boxes.append(d[:4])
                tracked_id.append(d[4])
#                 if j == 0:
#                     tracked_masks = mask
#                 else:
#                     tracked_masks = np.dstack([tracked_masks, mask])
    #             tracked_masks.append(mask_list[j])
                tracked_classes.append(d[-1])
                tracked_scores.append(d[-2])
                tracked_features.append(feature_list[j])

                # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
                f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
                            (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))
                
                
                
#             tracked_masks = np.reshape(tracked_masks, (tracked_masks.shape[0],tracked_masks.shape[1],j+1))
        else:
            for j, d in enumerate(trackers):
                tracked_boxes.append(d[:4])
                tracked_id.append(d[4])
                tracked_classes.append(d[-1])
                tracked_scores.append(d[-2])
                tracked_features.append(feature_list[j])
                # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
                f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
                            (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))
        
        tracked_boxes = np.array(tracked_boxes).astype('int')
        tracked_id = np.array(tracked_id)
        if len(tracked_id) == 0:
            continue
        tracked_classes = np.array(tracked_classes).astype('int')
        tracked_scores = np.array(tracked_scores)
#         if frame > 5:
#             break
    
#         save masked images
        save_path = out_path + '/' + str(format(frame,'04'))+'.png'
        masked_img = visualize.display_tracklets(img,
                                            tracked_boxes,
                                            tracked_id,
                                            tracked_masks, 
                                            tracked_classes, 
                                            class_names, 
                                            tracked_scores,
                                            show_mask = False,
                                            colors = colors,
                                            save_path = save_path)  # used only for display)
    tracked_features = np.array(tracked_features)
    print(tracked_features.shape)
    np.save(feature_out, tracked_features)
    
    
#         plt.clf()
        
#         save mask files
#         total_mask = np.zeros((640,1280),dtype=bool)
#         for i in range(tracked_masks.shape[2]):
#             total_mask = np.bitwise_or(total_mask, tracked_masks[:,:,i])

#         bbox_mask = np.ones((640,1280))
#         for box in tracked_boxes:
#             bbox_mask[box[0]:box[2], box[1]:box[3]] = 0
#         write_csv(out_path + str(format(frame,'04')) + '.csv' ,total_mask)
    
    print("One video is written!")
    f_out.close()
    break

In [ ]:
out_file

In [ ]:
tracked_boxes

In [ ]:
# import csv
# import cv2
# import numpy as np
# def read_binary(file_path,header=True,delimiter=','):
#     # The read-in data should be a N*W matrix,
#     # where N is the length of the time sequences,
#     # W is the number of sensors/data features
#     i = 0
#     with open(file_path, 'r') as file:
#         reader = csv.reader(file, delimiter = delimiter)
#         data=[]
#         for line in reader:
#             if i == 0 and header:
#                 i += +1
#             else:
#                 for j, element in enumerate(line):
#                     if element == 'True':
#                         line[j] = 0
#                     elif element == 'False':
#                         line[j] = 255
#                     else:
#                         raise ValueError("Data type is not boolean!!")
                    
#                 line = np.array(line) # str2float
#                 if i == 0  or (i == 1 and header):
#                     data = line
#                 else:
#                     data = np.vstack((data, line))
#                 i += 1
#     return data

In [ ]:
# # from data_reader import *
# masks = read_binary('/home/yyao/Documents/car_intersection/tracking_output/mask_rcnn/201804171444003136/0001.csv',header=False)

# RUN on 3D scene data

In [ ]:
# # del sys.modules['sort']
# # del sys.modules['mrcnn']
# from mrcnn import visualize
# from sort import Sort
# from utils import *
# import glob
# import time

# # only for testing
# # VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/samples/201806041123003053'
# VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/3D_data_samples/scenario_005/image_*.png'

# all_images = sorted(glob.glob(VIDEO_DIR))
# ROI = [0, 0, 1920, 1200]

# saver = True
# display = True
    
# colors = np.random.rand(32, 3)
# '''for saving observations of each video'''
# all_observations = {}


# '''for display'''
# if display:
#     colours = np.random.rand(32, 3)*255  # used only for display
#     plt.ion()
#     fig = plt.figure()
    
# '''init tracker'''
# use_dlibTracker = False # True to use dlib correlation tracker, False to use Kalman Filter tracker
# all_trackers =  Sort(ROI, max_age=1,min_hits=3, use_dlib=use_dlibTracker,track_masks=True)

# '''count time'''
# total_time = 0

# #     '''write results'''
# out_path = '/home/yyao/Documents/car_intersection/data/3D_data_samples/mrcnn_tracking/scenario_005/'
# try:
#     os.stat(out_path)
#     print("video has been processed!")
# #     continue
# except:# del sys.modules['sort']
# # del sys.modules['mrcnn']
# from mrcnn import visualize
# from sort import Sort
# from utils import *
# import glob
# import time

# # only for testing
# # VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/samples/201806041123003053'
# VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/3D_data_samples/scenario_005/image_*.png'

# all_images = sorted(glob.glob(VIDEO_DIR))
# ROI = [0, 0, 1920, 1200]

# saver = True
# display = True
    
# colors = np.random.rand(32, 3)
# '''for saving observations of each video'''
# all_observations = {}


# '''for display'''
# if display:
#     colours = np.random.rand(32, 3)*255  # used only for display
#     plt.ion()
#     fig = plt.figure()
    
# '''init tracker'''
# use_dlibTracker = False # True to use dlib correlation tracker, False to use Kalman Filter tracker
# all_trackers =  Sort(ROI, max_age=1,min_hits=3, use_dlib=use_dlibTracker,track_masks=True)

# '''count time'''
# total_time = 0

# #     '''write results'''
# out_path = '/home/yyao/Documents/car_intersection/data/3D_data_samples/mrcnn_tracking/scenario_005/'
# try:
#     os.stat(out_path)
#     print("video has been processed!")
# #     continue
# except:
#     os.mkdir(out_path)

# out_file = out_path + 'trackings.txt'
# f_out = open(out_file, 'w')


# frame = 0

# '''for saving observations of each car'''
# observations = {}
    
# for img in all_images:
#     img = cv2.imread(img)

#     if img is None:
#         break

#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


#     mrcnn_detections = model.detect([img], verbose=1)[0]

#     interesting_objects = np.where(mrcnn_detections['class_ids']<num_classes)[0]

#     bboxes = mrcnn_detections['rois'][interesting_objects]
#     masks = mrcnn_detections['masks'][:,:,interesting_objects]
#     classes = mrcnn_detections['class_ids'][interesting_objects]
#     scores = mrcnn_detections['scores'][interesting_objects]

#     frame += 1
#     start_time = time.time()
#     #update tracker
#     trackers, mask_list = all_trackers.update(bboxes.astype('int'),
#                                               img=img, 
#                                               masks=masks,
#                                               classes=classes,
#                                               scores=scores) # only cars are considered so far.



#     cycle_time = time.time() - start_time
#     total_time += cycle_time

#     print('frame: %d...took: %3fs'%(frame,cycle_time))

#     tracked_boxes = []
#     tracked_id = []
#     tracked_masks = []
#     tracked_classes = []
#     tracked_scores = []
#     for j, (d,mask) in enumerate(zip(trackers, mask_list)):
#         tracked_boxes.append(d[:4])
#         tracked_id.append(d[4])
#         if j == 0:
#             tracked_masks = mask
#         else:
#             tracked_masks = np.dstack([tracked_masks, mask])
# #             tracked_masks.append(mask_list[j])
#         tracked_classes.append(d[-1])
#         tracked_scores.append(d[-2])

#         # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
#         f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
#                     (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))

#     tracked_boxes = np.array(tracked_boxes).astype('int')
#     tracked_id = np.array(tracked_id)
#     print(tracked_id)
#     if len(tracked_id) == 0:
#         continue
#     tracked_masks = np.reshape(tracked_masks, (tracked_masks.shape[0],tracked_masks.shape[1],j+1))
#     tracked_classes = np.array(tracked_classes).astype('int')
#     tracked_scores = np.array(tracked_scores)
# del sys.modules['sort']
# # del sys.modules['mrcnn']
# from mrcnn import visualize
# from sort import Sort
# from utils import *
# import glob
# import time

# # only for testing
# # VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/samples/201806041123003053'
# VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/3D_data_samples/scenario_005/image_*.png'

# all_images = sorted(glob.glob(VIDEO_DIR))
# ROI = [0, 0, 1920, 1200]

# saver = True
# display = True
    
# colors = np.random.rand(32, 3)
# '''for saving observations of each video'''
# all_observations = {}


# '''for display'''
# if display:
#     colours = np.random.rand(32, 3)*255  # used only for display
#     plt.ion()
#     fig = plt.figure()
    
# '''init tracker'''
# use_dlibTracker = False # True to use dlib correlation tracker, False to use Kalman Filter tracker
# all_trackers =  Sort(ROI, max_age=1,min_hits=3, use_dlib=use_dlibTracker,track_masks=True)

# '''count time'''
# total_time = 0

# #     '''write results'''
# out_path = '/home/yyao/Documents/car_intersection/data/3D_data_samples/mrcnn_tracking/scenario_005/'
# try:
#     os.stat(out_path)
#     print("video has been processed!")
# #     continue
# except:
#     os.mkdir(out_path)

# out_file = out_path + 'trackings.txt'
# f_out = open(out_file, 'w')


# frame = 0

# '''for saving observations of each car'''
# observations = {}
    
# for img in all_images:
#     img = cv2.imread(img)

#     if img is None:
#         break

#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


#     mrcnn_detections = model.detect([img], verbose=1)[0]

#     interesting_objects = np.where(mrcnn_detections['class_ids']<num_classes)[0]

#     bboxes = mrcnn_detections['rois'][interesting_objects]
#     masks = mrcnn_detections['masks'][:,:,interesting_objects]
#     classes = mrcnn_detections['class_ids'][interesting_objects]
#     scores = mrcnn_detections['scores'][interesting_objects]

#     frame += 1
#     start_time = time.time()
#     #update tracker
#     trackers, mask_list = all_trackers.update(bboxes.astype('int'),
#                                               img=img, 
#                                               masks=masks,
#                                               classes=classes,
#                                               scores=scores) # only cars are considered so far.



#     cycle_time = time.time() - start_time
#     total_time += cycle_time

#     print('frame: %d...took: %3fs'%(frame,cycle_time))

#     tracked_boxes = []
#     tracked_id = []
#     tracked_masks = []
#     tracked_classes = []
#     tracked_scores = []
#     for j, (d,mask) in enumerate(zip(trackers, mask_list)):
#         tracked_boxes.append(d[:4])
#         tracked_id.append(d[4])
#         if j == 0:
#             tracked_masks = mask
#         else:
#             tracked_masks = np.dstack([tracked_masks, mask])
# #             tracked_masks.append(mask_list[j])
#         tracked_classes.append(d[-1])
#         tracked_scores.append(d[-2])

#         # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
#         f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
#                     (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))

#     tracked_boxes = np.array(tracked_boxes).astype('int')
#     tracked_id = np.array(tracked_id)
#     print(tracked_id)
#     if len(tracked_id) == 0:
#         continue
#     tracked_masks = np.reshape(tracked_masks, (tracked_masks.shape[0],tracked_masks.shape[1],j+1))
#     tracked_classes = np.array(tracked_classes).astype('int')
#     tracked_scores = np.array(tracked_scores)

#     save_path = out_path + str(format(frame,'04'))+'.png'
#     masked_img = visualize.display_tracklets(img,# del sys.modules['sort']
# # del sys.modules['mrcnn']
# from mrcnn import visualize
# from sort import Sort
# from utils import *
# import glob
# import time

# # only for testing
# # VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/samples/201806041123003053'
# VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/3D_data_samples/scenario_005/image_*.png'

# all_images = sorted(glob.glob(VIDEO_DIR))
# ROI = [0, 0, 1920, 1200]

# saver = True
# display = True
    
# colors = np.random.rand(32, 3)
# '''for saving observations of each video'''
# all_observations = {}


# '''for display'''
# if display:
#     colours = np.random.rand(32, 3)*255  # used only for display
#     plt.ion()
#     fig = plt.figure()
    
# '''init tracker'''
# use_dlibTracker = False # True to use dlib correlation tracker, False to use Kalman Filter tracker
# all_trackers =  Sort(ROI, max_age=1,min_hits=3, use_dlib=use_dlibTracker,track_masks=True)

# '''count time'''
# total_time = 0

# #     '''write results'''
# out_path = '/home/yyao/Documents/car_intersection/data/3D_data_samples/mrcnn_tracking/scenario_005/'
# try:
#     os.stat(out_path)
#     print("video has been processed!")
# #     continue
# except:
#     os.mkdir(out_path)

# out_file = out_path + 'trackings.txt'
# f_out = open(out_file, 'w')


# frame = 0

# '''for saving observations of each car'''
# observations = {}# del sys.modules['sort']
# # del sys.modules['mrcnn']
# from mrcnn import visualize
# from sort import Sort
# from utils import *
# import glob
# import time

# # only for testing
# # VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/samples/201806041123003053'
# VIDEO_DIR = '/home/yyao/Documents/car_intersection/data/3D_data_samples/scenario_005/image_*.png'

# all_images = sorted(glob.glob(VIDEO_DIR))
# ROI = [0, 0, 1920, 1200]

# saver = True
# display = True
    
# colors = np.random.rand(32, 3)
# '''for saving observations of each video'''
# all_observations = {}


# '''for display'''
# if display:
#     colours = np.random.rand(32, 3)*255  # used only for display
#     plt.ion()
#     fig = plt.figure()
    
# '''init tracker'''
# use_dlibTracker = False # True to use dlib correlation tracker, False to use Kalman Filter tracker
# all_trackers =  Sort(ROI, max_age=1,min_hits=3, use_dlib=use_dlibTracker,track_masks=True)

# '''count time'''
# total_time = 0

# #     '''write results'''
# out_path = '/home/yyao/Documents/car_intersection/data/3D_data_samples/mrcnn_tracking/scenario_005/'
# try:
#     os.stat(out_path)
#     print("video has been processed!")
# #     continue
# except:
#     os.mkdir(out_path)

# out_file = out_path + 'trackings.txt'
# f_out = open(out_file, 'w')


# frame = 0

# '''for saving observations of each car'''
# observations = {}
    
# for img in all_images:
#     img = cv2.imread(img)

#     if img is None:
#         break

#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


#     mrcnn_detections = model.detect([img], verbose=1)[0]

#     interesting_objects = np.where(mrcnn_detections['class_ids']<num_classes)[0]

#     bboxes = mrcnn_detections['rois'][interesting_objects]
#     masks = mrcnn_detections['masks'][:,:,interesting_objects]
#     classes = mrcnn_detections['class_ids'][interesting_objects]
#     scores = mrcnn_detections['scores'][interesting_objects]

#     frame += 1
#     start_time = time.time()
#     #update tracker
#     trackers, mask_list = all_trackers.update(bboxes.astype('int'),
#                                               img=img, 
#                                               masks=masks,
#                                               classes=classes,
#                                               scores=scores) # only cars are considered so far.



#     cycle_time = time.time() - start_time
#     total_time += cycle_time

#     print('frame: %d...took: %3fs'%(frame,cycle_time))

#     tracked_boxes = []
#     tracked_id = []
#     tracked_masks = []
#     tracked_classes = []
#     tracked_scores = []
#     for j, (d,mask) in enumerate(zip(trackers, mask_list)):
#         tracked_boxes.append(d[:4])
#         tracked_id.append(d[4])
#         if j == 0:
#             tracked_masks = mask
#         else:
#             tracked_masks = np.dstack([tracked_masks, mask])
# #             tracked_masks.append(mask_list[j])
#         tracked_classes.append(d[-1])
#         tracked_scores.append(d[-2])

#         # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
#         f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
#                     (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))

#     tracked_boxes = np.array(tracked_boxes).astype('int')
#     tracked_id = np.array(tracked_id)
#     print(tracked_id)
#     if len(tracked_id) == 0:
#         continue
#     tracked_masks = np.reshape(tracked_masks, (tracked_masks.shape[0],tracked_masks.shape[1],j+1))
#     tracked_classes = np.array(tracked_classes).astype('int')
#     tracked_scores = np.array(tracked_scores)

#     save_path = out_path + str(format(frame,'04'))+'.png'
#     masked_img = visualize.display_tracklets(img,
#                                         tracked_boxes,
#                                         tracked_id,
#                                         tracked_masks, 
#                                         tracked_classes, 
#                                         class_names, 
#                                         tracked_scores,
#                                         colors = colors,
#                                         save_path = save_path)  # used only for display)
#     plt.clf
#     plt.close

#     total_mask = np.zeros((1200,1920),dtype=bool)
#     for i in range(tracked_masks.shape[2]):
#         total_mask = np.bitwise_or(total_mask, tracked_masks[:,:,i])
# #         bbox_mask = np.ones((640,1280))
# #         for box in tracked_boxes:
# #             bbox_mask[box[0]:box[2], box[1]:box[3]] = 0
# #         write_csv(out_path + str(format(frame,'04')) + '.csv' ,total_mask)

# print("One video is written!")

    
# for img in all_images:
#     img = cv2.imread(img)

#     if img is None:
#         break

#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


#     mrcnn_detections = model.detect([img], verbose=1)[0]

#     interesting_objects = np.where(mrcnn_detections['class_ids']<num_classes)[0]

#     bboxes = mrcnn_detections['rois'][interesting_objects]
#     masks = mrcnn_detections['masks'][:,:,interesting_objects]
#     classes = mrcnn_detections['class_ids'][interesting_objects]
#     scores = mrcnn_detections['scores'][interesting_objects]

#     frame += 1
#     start_time = time.time()
#     #update tracker
#     trackers, mask_list = all_trackers.update(bboxes.astype('int'),
#                                               img=img, 
#                                               masks=masks,
#                                               classes=classes,
#                                               scores=scores) # only cars are considered so far.



#     cycle_time = time.time() - start_time
#     total_time += cycle_time

#     print('frame: %d...took: %3fs'%(frame,cycle_time))

#     tracked_boxes = []
#     tracked_id = []
#     tracked_masks = []
#     tracked_classes = []
#     tracked_scores = []
#     for j, (d,mask) in enumerate(zip(trackers, mask_list)):
#         tracked_boxes.append(d[:4])
#         tracked_id.append(d[4])
#         if j == 0:
#             tracked_masks = mask
#         else:
#             tracked_masks = np.dstack([tracked_masks, mask])
# #             tracked_masks.append(mask_list[j])
#         tracked_classes.append(d[-1])
#         tracked_scores.append(d[-2])

#         # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
#         f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
#                     (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))

#     tracked_boxes = np.array(tracked_boxes).astype('int')
#     tracked_id = np.array(tracked_id)
#     print(tracked_id)
#     if len(tracked_id) == 0:
#         continue
#     tracked_masks = np.reshape(tracked_masks, (tracked_masks.shape[0],tracked_masks.shape[1],j+1))
#     tracked_classes = np.array(tracked_classes).astype('int')
#     tracked_scores = np.array(tracked_scores)

#     save_path = out_path + str(format(frame,'04'))+'.png'
#     masked_img = visualize.display_tracklets(img,
#                                         tracked_boxes,
#                                         tracked_id,
#                                         tracked_masks, 
#                                         tracked_classes, 
#                                         class_names, 
#                                         tracked_scores,
#                                         colors = colors,
#                                         save_path = save_path)  # used only for display)
#     plt.clf
#     plt.close

#     total_mask = np.zeros((1200,1920),dtype=bool)
#     for i in range(tracked_masks.shape[2]):
#         total_mask = np.bitwise_or(total_mask, tracked_masks[:,:,i])
# #         bbox_mask = np.ones((640,1280))
# #         for box in tracked_boxes:
# #             bbox_mask[box[0]:box[2], box[1]:box[3]] = 0
# #         write_csv(out_path + str(format(frame,'04')) + '.csv' ,total_mask)

# print("One video is written!")

#                                         tracked_boxes,
#                                         tracked_id,
#                                         tracked_masks, 
#                                         tracked_classes, 
#                                         class_names, 
#                                         tracked_scores,
#                                         colors = colors,
#                                         save_path = save_path)  # used only for display)
#     plt.clf
#     plt.close

#     total_mask = np.zeros((1200,1920),dtype=bool)
#     for i in range(tracked_masks.shape[2]):
#         total_mask = np.bitwise_or(total_mask, tracked_masks[:,:,i])
# #         bbox_mask = np.ones((640,1280))
# #         for box in tracked_boxes:
# #             bbox_mask[box[0]:box[2], box[1]:box[3]] = 0
# #         write_csv(out_path + str(format(frame,'04')) + '.csv' ,total_mask)

# print("One video is written!")

#     save_path = out_path + str(format(frame,'04'))+'.png'
#     masked_img = visualize.display_tracklets(img,
#                                         tracked_boxes,
#                                         tracked_id,
#                                         tracked_masks, 
#                                         tracked_classes, 
#                                         class_names, 
#                                         tracked_scores,
#                                         colors = colors,
#                                         save_path = save_path)  # used only for display)
#     plt.clf
#     plt.close

#     total_mask = np.zeros((1200,1920),dtype=bool)
#     for i in range(tracked_masks.shape[2]):
#         total_mask = np.bitwise_or(total_mask, tracked_masks[:,:,i])
# #         bbox_mask = np.ones((640,1280))
# #         for box in tracked_boxes:
# #             bbox_mask[box[0]:box[2], box[1]:box[3]] = 0
# #         write_csv(out_path + str(format(frame,'04')) + '.csv' ,total_mask)

# print("One video is written!")

#     os.mkdir(out_path)

# out_file = out_path + 'trackings.txt'
# f_out = open(out_file, 'w')


# frame = 0

# '''for saving observations of each car'''
# observations = {}
    
# for img in all_images:
#     img = cv2.imread(img)

#     if img is None:
#         break

#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


#     mrcnn_detections = model.detect([img], verbose=1)[0]

#     interesting_objects = np.where(mrcnn_detections['class_ids']<num_classes)[0]

#     bboxes = mrcnn_detections['rois'][interesting_objects]
#     masks = mrcnn_detections['masks'][:,:,interesting_objects]
#     classes = mrcnn_detections['class_ids'][interesting_objects]
#     scores = mrcnn_detections['scores'][interesting_objects]

#     frame += 1
#     start_time = time.time()
#     #update tracker
#     trackers, mask_list = all_trackers.update(bboxes.astype('int'),
#                                               img=img, 
#                                               masks=masks,
#                                               classes=classes,
#                                               scores=scores) # only cars are considered so far.



#     cycle_time = time.time() - start_time
#     total_time += cycle_time

#     print('frame: %d...took: %3fs'%(frame,cycle_time))

#     tracked_boxes = []
#     tracked_id = []
#     tracked_masks = []
#     tracked_classes = []
#     tracked_scores = []
#     for j, (d,mask) in enumerate(zip(trackers, mask_list)):
#         tracked_boxes.append(d[:4])
#         tracked_id.append(d[4])
#         if j == 0:
#             tracked_masks = mask
#         else:
#             tracked_masks = np.dstack([tracked_masks, mask])
# #             tracked_masks.append(mask_list[j])
#         tracked_classes.append(d[-1])
#         tracked_scores.append(d[-2])

#         # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
#         f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
#                     (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))

#     tracked_boxes = np.array(tracked_boxes).astype('int')
#     tracked_id = np.array(tracked_id)
#     print(tracked_id)
#     if len(tracked_id) == 0:
#         continue
#     tracked_masks = np.reshape(tracked_masks, (tracked_masks.shape[0],tracked_masks.shape[1],j+1))
#     tracked_classes = np.array(tracked_classes).astype('int')
#     tracked_scores = np.array(tracked_scores)

#     save_path = out_path + str(format(frame,'04'))+'.png'
#     masked_img = visualize.display_tracklets(img,
#                                         tracked_boxes,
#                                         tracked_id,
#                                         tracked_masks, 
#                                         tracked_classes, 
#                                         class_names, 
#                                         tracked_scores,
#                                         colors = colors,
#                                         save_path = save_path)  # used only for display)
#     plt.clf
#     plt.close

#     total_mask = np.zeros((1200,1920),dtype=bool)
#     for i in range(tracked_masks.shape[2]):
#         total_mask = np.bitwise_or(total_mask, tracked_masks[:,:,i])
# #         bbox_mask = np.ones((640,1280))
# #         for box in tracked_boxes:
# #             bbox_mask[box[0]:box[2], box[1]:box[3]] = 0
# #         write_csv(out_path + str(format(frame,'04')) + '.csv' ,total_mask)

# print("One video is written!")
